In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

In [14]:
df = pd.read_csv("train_FD001.txt", sep = '\s+', header = None)
df1 = pd.read_csv("test_FD001.txt", sep = "\s+", header = None)

columns = [5,9,10,14,20,22,23]
train = df.drop(axis = 2, columns= columns)
test = df1.drop(axis=1, columns = columns)

In [15]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)
train_scaled = pd.DataFrame(train_scaled)
test_scaled = pd.DataFrame(test_scaled)

In [16]:
# define input sequence
sequence = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))

In [17]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

In [18]:
# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)

In [19]:
# lstm autoencoder recreate sequence
from numpy import array

# define input sequence
sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)
#plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

[0.10313464 0.20067136 0.2995478  0.39940697 0.4998805  0.6000924
 0.70008945 0.80001986 0.9001434 ]


In [20]:
train_scaled.shape[0]

20631

In [21]:

# define input sequence
seq_in = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))
# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
#plot_model(model, show_shapes=True, to_file='predict_lstm_autoencoder.png')
# fit model
model.fit(seq_in, seq_out, epochs=300, verbose=0)
# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat[0,:,0])

[0.16467692 0.28867602 0.40295145 0.50946975 0.61056083 0.7072067
 0.80013144 0.88996   ]


In [22]:
# define input sequence
seq_in = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))
# prepare output sequence
seq_out = seq_in[:, 1:, :]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
#plot_model(model, show_shapes=True, to_file='predict_lstm_autoencoder.png')
# fit model
model.fit(seq_in, seq_in, epochs=300, verbose=0)
# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat[0,:,0])

[0.1082323  0.20224571 0.29883194 0.39806294 0.49895656 0.6000171
 0.700791   0.80076665 0.8993319 ]


In [23]:
# define input sequence
seq_in = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))
# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

# define model
encoder = Sequential()
encoder.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
encoder.add(RepeatVector(n_in))

decoder = Sequential()
decoder.add(LSTM(100, activation='relu', return_sequences=True))
decoder.add((Dense(1)))

autoencoder = Sequential([encoder, decoder])
autoencoder.compile(optimizer='adam', loss='mse')

# fit model
model.fit(seq_in, seq_in, epochs=300, verbose=0)
# demonstrate prediction
seq = np.array([0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.5,1.6])
seq = seq.reshape((1,n_in,1))
yhat = model.predict(seq, verbose=0)
display(yhat[0,:,0])

array([0.16852173, 0.34556857, 0.54398865, 0.7573549 , 0.98636436,
       1.2347349 , 1.5083251 , 1.8159496 , 2.1706212 ], dtype=float32)

In [24]:
from keras.layers import LSTM
from keras.models import Sequential

# Encoder
encoder = Sequential()
encoder.add(LSTM(32, activation='relu', input_shape=(20,1)))

# Decoder 
decoder = Sequential() 
decoder.add(LSTM(32, activation='relu', input_shape=(32,)))
decoder.add(Dense(20)) 

# Autoencoder
autoencoder = Sequential([encoder, decoder])
autoencoder.compile(loss='mse', optimizer='adam')

ValueError: Input 0 of layer "lstm_21" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 32)